In [2]:
import pandas as pd

In [4]:
endangeredSpecies = pd.read_csv('endangeredSpecies.csv')
endangeredSpecies.head()

,identificacion,nombre_cientifico,reino,filo,clase,orden,familia,genero,epiteto_especifico,estado_taxonomico,estado_de_amenaza,coordenadas
0,3548724,Arthonia obscurella,Fungi,Ascomycota,Arthoniomycetes,Arthoniales,Arthoniaceae,Arthonia,obscurella,Aceptado,CR,"[(4.59806, -74.07583)]"
1,3548779,Arthonia septemlocularis,Fungi,Ascomycota,Arthoniomycetes,Arthoniales,Arthoniaceae,Arthonia,septemlocularis,Aceptado,CR,"[(4.59806, -74.07583)]"
2,10800176,Ancistrosporella leucophila,Fungi,Ascomycota,Arthoniomycetes,Arthoniales,Roccellaceae,Ancistrosporella,leucophila,Aceptado,CR,"[(6.98931, -73.04892), (4.0, -72.0), (6.98701,..."
3,10874088,Byssoloma permutans,Fungi,Ascomycota,Lecanoromycetes,Lecanorales,Byssolomataceae,Byssoloma,permutans,Aceptado,CR,[]
4,5516240,Lecanactis proximans,Fungi,Ascomycota,Arthoniomycetes,Arthoniales,Roccellaceae,Lecanactis,proximans,Aceptado,CR,"[(4.0, -72.0), (4.0, -72.0), (4.645293, -74.06..."


In [5]:
endangeredAmphibiaSpecies = endangeredSpecies.loc[endangeredSpecies['clase'] == 'Amphibia']
endangeredAmphibiaSpecies.head()

,identificacion,nombre_cientifico,reino,filo,clase,orden,familia,genero,epiteto_especifico,estado_taxonomico,estado_de_amenaza,coordenadas
1526,2431315,Bolitoglossa capitana,Animalia,Chordata,Amphibia,Caudata,Plethodontidae,Bolitoglossa,capitana,Válido,CR,"[(4.53148, -74.336), (4.53148, -74.336), (4.53..."
1527,9338198,Bolitoglossa guaneae,Animalia,Chordata,Amphibia,Caudata,Plethodontidae,Bolitoglossa,guaneae,Válido,VU,"[(5.617806, -73.956917), (5.617694, -73.956806..."
1528,2431289,Bolitoglossa hiemalis,Animalia,Chordata,Amphibia,Caudata,Plethodontidae,Bolitoglossa,hiemalis,Válido,VU,"[(4.063167, -76.499361), (4.063167, -76.499361..."
1529,2431296,Bolitoglossa hypacra,Animalia,Chordata,Amphibia,Caudata,Plethodontidae,Bolitoglossa,hypacra,Válido,EN,"[(6.453389, -76.099083), (6.453389, -76.099083..."
1530,9577531,Bolitoglossa leandrae,Animalia,Chordata,Amphibia,Caudata,Plethodontidae,Bolitoglossa,leandrae,Válido,CR,"[(4.86, -73.26), (4.86, -73.26), (4.15, -73.65)]"


In [17]:
endangeredAmphibiaSpecies.to_csv('endangeredAmphibiaSpecies.csv', index=False)

In [8]:
import pandas as pd
import folium

# Read the CSV data into a pandas DataFrame
df = pd.read_csv('endangeredAmphibiaSpecies.csv')

# Create a base map with dark theme
m = folium.Map(location=[4.5709, -74.2973], zoom_start=8, tiles='CartoDB dark_matter')

# Create a feature group for each conservation status
threatened_fg = folium.FeatureGroup(name='Threatened Species')
endangered_fg = folium.FeatureGroup(name='Endangered Species')
critical_fg = folium.FeatureGroup(name='Critically Endangered Species')

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    species = row['nombre_cientifico']
    status = row['estado_de_amenaza']
    location = row['display_name']
    coordinates = eval(row['coordenadas'])  # Convert string to list of tuples

    # Create a popup with the species name and conservation status
    popup_text = f"<b>{species}</b><br>Conservation Status: {status}</b><br>Location: {location}</b>"

    # Add circles to the appropriate feature group based on the conservation status
    if status == 'VU':
        for coord in coordinates:
            folium.Circle(location=coord, radius=250, popup=popup_text, color='darkblue', fill=True,
                          fill_opacity=0.5).add_to(threatened_fg)
    elif status == 'EN':
        for coord in coordinates:
            folium.Circle(location=coord, radius=250, popup=popup_text, color='orange', fill=True,
                          fill_opacity=0.5).add_to(endangered_fg)
    elif status == 'CR':
        for coord in coordinates:
            folium.Circle(location=coord, radius=250, popup=popup_text, color='red', fill=True,
                          fill_opacity=0.5).add_to(critical_fg)

# Add the feature groups to the map
threatened_fg.add_to(m)
endangered_fg.add_to(m)
critical_fg.add_to(m)

# Add a layer control to toggle between feature groups
folium.LayerControl().add_to(m)

# Add a legend with matching colors
legend_html = '''
    <div style="position: fixed; bottom: 50px; left: 50px; background-color: rgba(255, 255, 255, 0.8); padding: 15px; border-radius: 5px; box-shadow: 0 0 10px rgba(0, 0, 0, 0.3); z-index: 9999; font-family: Arial, sans-serif; font-size: 14px;">
    <h3 style="margin-top: 0;">Amphibia Species Endangered</h3>
    <div style="display: flex; align-items: center; margin-bottom: 5px;">
        <i class="fa fa-circle" style="color: red; margin-right: 5px;"></i>
        <span>Critically Endangered</span>
    </div>
    <div style="display: flex; align-items: center; margin-bottom: 5px;">
        <i class="fa fa-circle" style="color: orange; margin-right: 5px;"></i>
        <span>Endangered</span>
    </div>
    <div style="display: flex; align-items: center;">
        <i class="fa fa-circle" style="color: darkblue; margin-right: 5px;"></i>
        <span>Threatened</span>
    </div>
</div>
    '''
m.get_root().html.add_child(folium.Element(legend_html))

# Display the map
m

# Save the map as an HTML file
m.save('endangered_amphibia_map.html')

In [30]:
endangeredAmphibiaSpecies.loc[endangeredSpecies['estado_de_amenaza'] == 'CR']

,identificacion,nombre_cientifico,reino,filo,clase,orden,familia,genero,epiteto_especifico,estado_taxonomico,estado_de_amenaza,coordenadas
1526,2431315,Bolitoglossa capitana,Animalia,Chordata,Amphibia,Caudata,Plethodontidae,Bolitoglossa,capitana,Válido,CR,"[(4.53148, -74.336), (4.53148, -74.336), (4.53..."
1530,9577531,Bolitoglossa leandrae,Animalia,Chordata,Amphibia,Caudata,Plethodontidae,Bolitoglossa,leandrae,Válido,CR,"[(4.86, -73.26), (4.86, -73.26), (4.15, -73.65)]"
1539,2426290,Allobates ranoides,Animalia,Chordata,Amphibia,Anura,Aromobatidae,Allobates,ranoides,Válido,CR,"[(4.773, -73.038002), (4.773, -73.038002), (4...."
1540,5216705,Atelopus angelito,Animalia,Chordata,Amphibia,Anura,Bufonidae,Atelopus,angelito,Válido,CR,"[(5.08444, -75.29357)]"
1541,7785931,Atelopus ardila,Animalia,Chordata,Amphibia,Anura,Bufonidae,Atelopus,ardila,Válido,CR,[]
...,...,...,...,...,...,...,...,...,...,...,...,...
1785,5218147,Gastrotheca angustifrons,Animalia,Chordata,Amphibia,Anura,Hemiphractidae,Gastrotheca,angustifrons,Válido,CR,[]
1786,5218196,Gastrotheca antomia,Animalia,Chordata,Amphibia,Anura,Hemiphractidae,Gastrotheca,antomia,Válido,CR,"[(4.75305, -76.2911), (4.75305, -76.2911), (4...."
1801,2428303,Hyloscirtus lynchi,Animalia,Chordata,Amphibia,Anura,Hylidae,Hyloscirtus,lynchi,Válido,CR,"[(7.347478, -72.888798), (7.570184, -72.842487..."
1802,2428289,Hyloscirtus pantostictus,Animalia,Chordata,Amphibia,Anura,Hylidae,Hyloscirtus,pantostictus,Válido,CR,[]


# Using the Nominatim API

In [21]:
endangeredAmphibiaSpecies = pd.read_csv('endangeredAmphibiaSpecies.csv')
endangeredAmphibiaSpecies.head()

,identificacion,nombre_cientifico,reino,filo,clase,orden,familia,genero,epiteto_especifico,estado_taxonomico,estado_de_amenaza,coordenadas,location_info,display_name,town,state_district,country
0,2431315,Bolitoglossa capitana,Animalia,Chordata,Amphibia,Caudata,Plethodontidae,Bolitoglossa,capitana,Válido,CR,"[(4.53148, -74.336), (4.53148, -74.336), (4.53...","{'place_id': 298208485, 'licence': 'Data © Ope...","Granada, Sumapaz, RAP (Especial) Central, Colo...",Granada,Sumapaz,Colombia
1,9338198,Bolitoglossa guaneae,Animalia,Chordata,Amphibia,Caudata,Plethodontidae,Bolitoglossa,guaneae,Válido,VU,"[(5.617806, -73.956917), (5.617694, -73.956806...","{'place_id': 298487144, 'licence': 'Data © Ope...","Llano Grande, Pauna, Occidente, Boyacá, RAP (E...",NaN,Occidente,Colombia
2,2431289,Bolitoglossa hiemalis,Animalia,Chordata,Amphibia,Caudata,Plethodontidae,Bolitoglossa,hiemalis,Válido,VU,"[(4.063167, -76.499361), (4.063167, -76.499361...","{'place_id': 298065881, 'licence': 'Data © Ope...","Riofrío, Centro, Valle del Cauca, RAP Pacífico...",NaN,Centro,Colombia
3,2431296,Bolitoglossa hypacra,Animalia,Chordata,Amphibia,Caudata,Plethodontidae,Bolitoglossa,hypacra,Válido,EN,"[(6.453389, -76.099083), (6.453389, -76.099083...","{'place_id': 298122804, 'licence': 'Data © Ope...","Sendero a Campanas, Urrao, Suroeste, Antioquia...",Urrao,Suroeste,Colombia
4,9577531,Bolitoglossa leandrae,Animalia,Chordata,Amphibia,Caudata,Plethodontidae,Bolitoglossa,leandrae,Válido,CR,"[(4.86, -73.26), (4.86, -73.26), (4.15, -73.65)]","{'place_id': 298066345, 'licence': 'Data © Ope...","Marginal de la Selva, Santa María, Neira, Boya...",Santa María,Neira,Colombia


In [10]:
import requests
from json import JSONDecodeError
import ast


def get_location_info(lat, lon):
    url = f'https://nominatim.openstreetmap.org/reverse?format=json&lat={lat}&lon={lon}'
    headers = {
        'User-Agent': 'Mozilla/5.0 (compatible; your_app_name/0.1; +http://yourwebsite.com/bot)'
    }
    response = requests.get(url, headers=headers)

    # Check if the response status code is OK
    if response.status_code != 200:
        return None

    # Try to decode the JSON response
    try:
        return response.json()
    except JSONDecodeError:
        return None

In [11]:
# Function to extract the first coordinate and get location info
def extract_first_coordinate(coord_str):
    try:
        # Convert string representation of list to actual list
        coords = ast.literal_eval(coord_str)
        if coords:
            first_coord = coords[0]
            lat, lon = first_coord
            location_info = get_location_info(lat, lon)
            return location_info
        return None
    except (ValueError, SyntaxError):
        return None

In [12]:
# Apply the function to the DataFrame
endangeredAmphibiaSpecies['location_info'] = endangeredAmphibiaSpecies['coordenadas'].apply(extract_first_coordinate)

In [13]:
endangeredAmphibiaSpecies['location_info']

0      {'place_id': 298208485, 'licence': 'Data © Ope...
1      {'place_id': 298487144, 'licence': 'Data © Ope...
2      {'place_id': 298065881, 'licence': 'Data © Ope...
3      {'place_id': 298122804, 'licence': 'Data © Ope...
4      {'place_id': 298066345, 'licence': 'Data © Ope...
                             ...                        
283                                                 None
284                                                 None
285    {'place_id': 269678686, 'licence': 'Data © Ope...
286                                                 None
287    {'place_id': 269699366, 'licence': 'Data © Ope...
Name: location_info, Length: 288, dtype: object

In [20]:

# Function to extract specific keys from the location_info dictionary
def extract_location_info(info, key):
    if info and key in info:
        return info[key]
    return None


def extract_address_info(info, key):
    if info and 'address' in info and key in info['address']:
        return info['address'][key]
    return None


# Create new columns for display_name, town, state_district, and country
endangeredAmphibiaSpecies['display_name'] = endangeredAmphibiaSpecies['location_info'].apply(
    lambda x: extract_location_info(x, 'display_name'))
endangeredAmphibiaSpecies['town'] = endangeredAmphibiaSpecies['location_info'].apply(
    lambda x: extract_address_info(x, 'town'))
endangeredAmphibiaSpecies['state_district'] = endangeredAmphibiaSpecies['location_info'].apply(
    lambda x: extract_address_info(x, 'state_district'))
endangeredAmphibiaSpecies['country'] = endangeredAmphibiaSpecies['location_info'].apply(
    lambda x: extract_address_info(x, 'country'))

In [16]:
endangeredAmphibiaSpecies

In [19]:
endangeredAmphibiaSpecies.town.value_counts()

Urrao                       6
El Carmen de Chucurí        6
Pensilvania                 5
Ciénaga                     5
Samaná                      4
Dabeiba                     4
Anorí                       3
Tadó                        3
Santa Bárbara               2
Aracataca                   2
Vetas                       2
Unguía                      2
Cajamarca                   2
Betania                     2
Charalá                     2
Facatativá                  1
Floridablanca               1
Valdivia                    1
Tona                        1
Villa Caro                  1
Guatapé                     1
San Francisco               1
Bahía Solano (Mutis)        1
Ubalá                       1
Cimitarra                   1
Calima                      1
Piedecuesta                 1
Puerto López                1
Silvania                    1
Marquetalia                 1
Palocabildo                 1
San José de la Montaña      1
La Ceja                     1
Granada   

In [20]:
endangeredAmphibiaSpecies.country.value_counts()

Colombia     192
Panamá         4
Venezuela      1
Name: country, dtype: int64

In [4]:
endangeredAmphibiaSpecies.state_district.value_counts()

Norte                   20
Centro                  17
Oriente                 16
Suroeste                15
Yariguíes                9
Occidente                9
Vertiente Pacífico       8
Alto Oriente             6
Nordeste                 5
Guavio                   5
Atrato                   5
Magdalena Caldense       4
Metropolitana            4
San Juan                 4
Sumapaz                  3
Nevados                  3
Piedemonte               3
Lengupá                  2
Comunera                 2
Guanentá                 2
Centrosur                2
Sabana Occidente         2
Soto Norte               2
Gualivá                  2
Fría                     2
Tequendama               1
Valle de Aburrá          1
Rio Meta                 1
Vertiente Occidental     1
Nororiente               1
Márquez                  1
Ricaurte                 1
Tundama                  1
La Libertad              1
Neira                    1
Pacífico Norte           1
Name: state_district, dtype:

In [22]:
endangeredAmphibiaSpecies.display_name.value_counts()

El Cairo, Norte, Valle del Cauca, RAP Pacífico, Colombia                                                                                                                     11
El Carmen de Atrato, Atrato, Chocó, RAP Pacífico, Colombia                                                                                                                    4
Reserva Natural Fundación Bioconservancy, Por el lecho del río después de avalancha 13ENE23, La Mesenia, Jardín, Suroeste, Antioquia, RAP del Agua y la Montaña, Colombia     4
Santa Marta, Magdalena, RAP Caribe, Colombia                                                                                                                                  4
Riofrío, Centro, Valle del Cauca, RAP Pacífico, Colombia                                                                                                                      4
                                                                                                                        